In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import OLSInfluence as influence
pd.set_option('display.max_columns', None)

In [ ]:
#read in the file
df = pd.read_csv('data/kc_house_data.csv')

In [ ]:
#check the first 5 entries in the data
df.head()

In [ ]:
#check the columns and nulls
df.info()

In [ ]:
#for year renovated, convert any houses that have been renovated to '1' to indicate true
#for any nulls, assume no renovation
df['yr_renovated'].fillna(0, inplace=True)
df['yr_renovated'] = df['yr_renovated'].apply(lambda x: 1 if x > 0 else x)

In [ ]:
df.rename(columns={'yr_renovated': 'if_renovated'}, inplace=True)

In [ ]:
#clean up sqft_basement and convert to int
df['sqft_basement'] = df['sqft_basement'].replace({'?':np.nan}).astype(float)
df['sqft_basement'].fillna(df['sqft_living']-df['sqft_above'], inplace=True)

In [ ]:
#retrieve the months and year
df['month_of_date'] = pd.DatetimeIndex(df['date']).month
df['year_of_date'] = pd.DatetimeIndex(df['date']).year

In [ ]:
#convert yr_built to age of house by subtracting year the property was sold by the year it was built
#to create a more sensible column 
df['age_of_house'] = df['year_of_date'] - df['yr_built']

#drop year of date because years are only 2014 and 2015, and will not impact our predicative model
#drop yr_built b/c it is redundant with age_of_house
df.drop(columns=['year_of_date'], inplace=True)
df.drop(columns=['yr_built'], inplace=True)

In [ ]:
#drop duplicates if any
df.drop_duplicates(inplace=True)

In [ ]:
#drop id and date columns
df.drop(columns=['id'], inplace=True)
df.drop(columns=['date'], inplace=True)

In [ ]:
#reset index
df.reset_index(inplace=True, drop=True)

In [ ]:
#convert some of the categorical data from floats to ints
df['waterfront'] = df['waterfront'].astype(int)
df['view'] = df['view'].astype(int)
df['sqft_basement'] = df['sqft_basement'].astype(int)
df['if_renovated'] = df['if_renovated'].astype(int)

In [ ]:
#check cleaned data
df.info()

In [ ]:
df.head()

# Exploratory Data Analysis (EDA)

In [ ]:
#Since price is our target, we will explore first
#view distribution of price using histogram
sns.set(style = 'white')
fig, ax = plt.subplots(figsize = (8,8))
sns.histplot(data = df, x = 'price', palette = 'Dark', )
ax.set_xlabel('Price')
ax.set_ylabel('Count')
ax.set_title('Distribution of Price')
plt.show()

In [ ]:
#plot a boxplot for price
sns.set(style = 'white')
fig, ax = plt.subplots(figsize = (6,6))
sns.boxplot(data = df, x = 'price', palette = 'pastel', fliersize = 5, whis = 8)
ax.set_xlabel('Price')
ax.set_ylabel('Count')
ax.set_title('Distribution of Price')
plt.show()

In [ ]:
#Because the data is skewed to the right, transform the price data using log
df['ln_price'] = np.log(df['price'])

In [ ]:
#view distribution of log base e for price using histogram
sns.set(style = 'white')
fig, ax = plt.subplots(figsize = (6,6))
sns.histplot(data = df, x = 'ln_price', palette = 'Dark')
ax.set_xlabel('Natural Log of Price')
ax.set_ylabel('Count')
ax.set_title('Distribution of Natural Log of Price')
plt.show()

In [ ]:
sns.set(style = 'white')
fig, ax = plt.subplots(figsize = (6,6))
sns.boxplot(data = df, x = 'ln_price', palette = "pastel")
ax.set_xlabel("Price")
ax.set_ylabel("Count")
ax.set_title("Distribution of Price")
plt.show()

### Features

In [ ]:
df.columns

In [ ]:
#based on the pairplot, we can see which data are categorical and which are numeric
numeric = ['bedrooms', 
           'bathrooms', 
           'sqft_living', 
           'sqft_lot', 
           'sqft_above', 
           'sqft_basement',
           'lat', 
           'long',
           'sqft_living15', 
           'sqft_lot15']

categorical = ['floors',
               'waterfront', 
               'view', 
               'condition', 
               'grade',
               'if_renovated',
               'zipcode',
               'month_of_date']

In [ ]:

#visually inspecting value counts to look for weird values
for column in df.columns:
    if not column == 'price':
        display(df[column].value_counts())

In [ ]:
#found 1 obs that should be removed
df[df['bedrooms'] >= 20]
df.drop(15856, inplace=True)

In [ ]:
#these look fine
df[df['bathrooms']>6]

In [ ]:
# Create a df with the target as the first column,
# then compute the correlation matrix
X = df[numeric]
y = df['price']
heatmap_data = pd.concat([y, X], axis=1)
corr = heatmap_data.corr()

# Set up figure and axes
fig, ax = plt.subplots(figsize=(15, 15))

# Plot a heatmap of the correlation matrix, with both
# numbers and colors indicating the correlations
sns.heatmap(
    # Specifies the data to be plotted
    data=corr,
    # The mask means we only show half the values,
    # instead of showing duplicates. It's optional.
    mask=np.triu(np.ones_like(corr, dtype=bool)),
    # Specifies that we should use the existing axes
    ax=ax,
    # Specifies that we want labels, not just colors
    annot=True,
    # Customizes colorbar appearance
    cbar_kws={"label": "Correlation", "orientation": "horizontal", "pad": .2, "extend": "both"}
)

# Customize the plot appearance
ax.set_title("Heatmap of Correlation Between Attributes and Price");

In [ ]:
#reporting the correlation between price (target) and predictors
df.corr()['price'].drop(['ln_price']).map(abs).sort_values(ascending=False)

In [ ]:
# Create a df with the target as the first column,
# then compute the correlation matrix
X = df[numeric]
ln_y = df['ln_price']
heatmap_data = pd.concat([ln_y, X], axis=1)
corr = heatmap_data.corr()

# Set up figure and axes
fig, ax = plt.subplots(figsize=(15, 15))

# Plot a heatmap of the correlation matrix, with both
# numbers and colors indicating the correlations
sns.heatmap(
    # Specifies the data to be plotted
    data=corr,
    # The mask means we only show half the values,
    # instead of showing duplicates. It's optional.
    mask=np.triu(np.ones_like(corr, dtype=bool)),
    # Specifies that we should use the existing axes
    ax=ax,
    # Specifies that we want labels, not just colors
    annot=True,
    # Customizes colorbar appearance
    cbar_kws={"label": "Correlation", "orientation": "horizontal", "pad": .2, "extend": "both"}
)

# Customize the plot appearance
ax.set_title("Heatmap of Correlation Between Attributes and Price");

In [ ]:
#reporting the correlation between ln price (target) and predictors
df.corr()['ln_price'].drop(['price']).map(abs).sort_values(ascending=False)

In [ ]:
#plot the numeric features to see distribution and see if they need to be log-transformed
df[numeric].hist(figsize=[6,6]);
plt.tight_layout()
plt.show()

In [ ]:
#log transform all numeric feature except:
#sqft basement - has values of 0
#long - has negative values
to_ln = ['bedrooms',
         'bathrooms',
         'sqft_living',
         'sqft_lot', 
         'sqft_above',
         'lat',
         'sqft_living15', 
         'sqft_lot15']

for column in to_ln:
    df[column] = np.log(df[column])

# Data Mainipulation

In [ ]:
#create two dataframes, one without ln_price, and without price
output = df.drop(['ln_price'], axis=1) 
output_ln = df.drop(['price'], axis=1) 

In [ ]:
def train_test(df, target, test_size=0.20, random_state=42):
    '''
    This function takes in a dataframe df and target column and returns the train and test split
    Default test size is 20, default random state is 42
    '''
    
    #dropping targets out of predictors
    X = df.drop(target, axis=1)

    #set target with y
    y = df[target]
    
    #creating  train test split for model comparison
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    #instantiating OHE object
    ohe = OneHotEncoder(sparse=False, handle_unknown='error', drop='first')

    #Fitting object onto test and trasnforming test and train
    X_train_ohe = ohe.fit_transform(X_train[categorical])
    X_test_ohe = ohe.transform(X_test[categorical])

    #placing column names onto our new categorical columns and formatting as df
    X_train_ohe_df = pd.DataFrame(X_train_ohe, columns=ohe.get_feature_names(categorical), 
                              index=X_train.index)
    X_test_ohe_df = pd.DataFrame(X_test_ohe, columns=ohe.get_feature_names(categorical),
                            index=X_test.index)

    #combining categoricals with rest of data
    X_train = pd.concat([X_train[numeric], X_train_ohe_df],axis=1)
    X_test = pd.concat([X_test[numeric], X_test_ohe_df], axis=1)

    
    X_list = [X_train, X_test]
    
    #scaling X values into z-scores
    ss = StandardScaler()
    for i in X_list:
        ss.fit(i)
        i = pd.DataFrame(ss.transform(i))
        
    return X_train, X_test, y_train, y_test

### Other Formulas and Useful Objects

In [ ]:
def cross_val(estimator,X,y,n_splits=10,test_size=0.25, random_state=None):
    """
    This formula performs cross validation using shuffled splits. Output is a tuple,
    The 0th element is the median R2 score for the train sets, the 1st element
    is the median R2 score for the test sets.
    
    """
    splitter = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)

    output = cross_validate(estimator, X=X, y=y, cv=splitter, return_train_score=True)
    return np.median(output['train_score']), np.median(output['test_score'])

In [ ]:
#returns a summary of the median train R-squared, median test R-squared, and differential score based
#on the cross validation
def cval_summary(train,test,diff):
    return f"The median R-squared values for the train sets were {round(train,3)}, the median R-squared values for the test sets were {round(test,3)}. The

In [ ]:
def score_model(model, Xtrain, Xtest, ytrain, ytest, log=False):
    '''
    This function takes in a model and the train and test samples and returns
    the train R-squared, test R-squared, and the RMSE
    '''
    if log == False:
        rmse = mean_squared_error(ytest, model.predict(Xtest), squared=False)
    else:
        rmse = mean_squared_error(np.exp(ytest), np.exp(model.predict(Xtest)), squared=False)
    return model.score(Xtrain, ytrain),  model.score(Xtest, ytest), rmse


In [ ]:
#returns a summary of the train R-squared, test R-squared, differential between R-squared, and RMSE
def model_summary(train,test,diff,rmse):
    return f"The R-squared value for the train set was {round(train,3)}, and the R-squared value for the test set was {round(test,3)}. These values resulted in a differential of {round(diff,5)}. The RMSE of our model predicitons was {round(rmse,2)}"